get the dataset from coco-2017
the dataset is person only
train = 3000
validation = 500

In [ ]:
import tensorflow as tf
import fiftyone as fo
import fiftyone.zoo as foz

# Load the COCO-2017 dataset
# This will download it from the FiftyOne Dataset Zoo if necessary
dataset = foz.load_zoo_dataset("coco-2017", split="train", label_types=["detections"], classes=["person"], max_samples=3000)
dataset_test = foz.load_zoo_dataset("coco-2017", split="validation", label_types=["detections"], classes=["person"], max_samples=500)


Found annotations at 'C:\Users\angga\fiftyone\coco-2017\raw\instances_train2017.json'
Sufficient images already downloaded
Existing download of split 'train' is sufficient
Loading existing dataset 'coco-2017-train-300'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Found annotations at 'C:\Users\angga\fiftyone\coco-2017\raw\instances_val2017.json'
Sufficient images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'coco-2017-validation-50'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


Explore the dataset

In [ ]:
print(dataset)
print(dataset.first())

Name:        coco-2017-train-300
Media type:  image
Num samples: 300
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
<Sample: {
    'id': '64c7ce84735d977371ab6696',
    'media_type': 'image',
    'filepath': 'C:\\Users\\angga\\fiftyone\\coco-2017\\train\\data\\000000000036.jpg',
    'tags': ['train'],
    'metadata': <ImageMetadata: {
        'size_bytes': None,
        'mime_type': None,
        'width': 481,
        'height': 640,
        'num_channels': None,
    }>,
    'ground_truth': <Detections: {
        'detections': [
            <Detection: {
                'id': '64c7ce84735d977371ab6694',
       

get the image path and annotations.json
then convert the json format to xml format

In [ ]:
import xml.etree.ElementTree as ET
import json
import os
import xmltodict

def json_to_xml(json_path, xml_path, image_folder_path, target_label="person"):
    with open(json_path, 'r') as f:
        data = json.load(f)

    images = data['images']
    annotations = data['annotations']
    categories = data['categories']

    image_id_to_filename = {image['id']: image['file_name'] for image in images}
    image_id_to_annotations = {image['id']: [] for image in images}

    for ann in annotations:
        image_id = ann['image_id']
        image_id_to_annotations[image_id].append(ann)

    for image_id, filename in image_id_to_filename.items():
        image_path = os.path.join(image_folder_path, filename)
        image_annotations = image_id_to_annotations[image_id]

        root = ET.Element("annotation")

        folder = ET.SubElement(root, "folder").text = image_folder_path
        filename_xml = ET.SubElement(root, "filename").text = filename

        size = ET.SubElement(root, "size")
        width = ET.SubElement(size, "width").text = str(data['images'][0]['width'])
        height = ET.SubElement(size, "height").text = str(data['images'][0]['height'])
        depth = ET.SubElement(size, "depth").text = str(3)  # Assuming 3 channels for RGB images

        for ann in image_annotations:
            category_id = ann['category_id']
            category_name = [cat['name'] for cat in categories if cat['id'] == category_id][0]
            bbox = ann['bbox']

            if category_name == target_label:
                object_xml = ET.SubElement(root, "object")
                name = ET.SubElement(object_xml, "name").text = category_name
                bndbox = ET.SubElement(object_xml, "bndbox")
                xmin = ET.SubElement(bndbox, "xmin").text = str(bbox[0])
                ymin = ET.SubElement(bndbox, "ymin").text = str(bbox[1])
                xmax = ET.SubElement(bndbox, "xmax").text = str(bbox[0] + bbox[2])
                ymax = ET.SubElement(bndbox, "ymax").text = str(bbox[1] + bbox[3])

        if len(root.findall("object")) > 0:  # Only save the annotation if there is at least one "person" object
            tree = ET.ElementTree(root)
            tree.write(os.path.join(xml_path, os.path.splitext(filename)[0] + ".xml"))

# Example usage:
json_to_xml("train/labels.json", "train_xml/", "train/data/", target_label="person")
json_to_xml("validation/labels.json", "validation_xml/", "validation/data/", target_label="person")


verify the xml files with dataframe preview

In [ ]:
import glob
import xml.etree.ElementTree as ET
import pandas as pd

def parse_xml_files(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (
                root.find('filename').text,
                int(root.find('size/width').text),
                int(root.find('size/height').text),
                member.find('name').text,
                int(float(member.find('bndbox/xmin').text)),
                int(float(member.find('bndbox/ymin').text)),
                int(float(member.find('bndbox/xmax').text)),
                int(float(member.find('bndbox/ymax').text))
            )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

# Usage example:
path_to_xml_files = './Tensorflow/workspace/images/train'
xml_df = parse_xml_files(path_to_xml_files)
print(xml_df)

create label map as person only with format protobuf txt

In [ ]:
label_map = {
    'person': 1  # Add other classes if needed
}


Create to tf.record format

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME),
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'),
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'),
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
labels = [{'name':'person', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')}
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')}

re-check the labelling

In [ ]:
import os
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

def read_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    objects = root.findall('object')
    labels = []
    bboxes = []
    for obj in objects:
        label = obj.find('name').text
        bbox = obj.find('bndbox')
        xmin = float(bbox.find('xmin').text)
        ymin = float(bbox.find('ymin').text)
        xmax = float(bbox.find('xmax').text)
        ymax = float(bbox.find('ymax').text)
        labels.append(label)
        bboxes.append((xmin, ymin, xmax, ymax))
    return labels, bboxes

def plot_image_with_bbox(image_path, xml_path):
    image = Image.open(image_path)
    labels, bboxes = read_xml(xml_path)

    fig, ax = plt.subplots(1)
    ax.imshow(image)

    for bbox, label in zip(bboxes, labels):
        xmin, ymin, xmax, ymax = bbox
        width = xmax - xmin
        height = ymax - ymin
        rect = patches.Rectangle((xmin, ymin), width, height, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.text(xmin, ymin, label, bbox=dict(facecolor='red', alpha=0.5))

    plt.show()

# Replace '000000000036.jpg' with the name of your image file
image_path = './train/data/000000000061.jpg'
# Replace 'train.xml' with the path to your train.xml file
xml_path = './train_xml/000000000061.xml'

plot_image_with_bbox(image_path, xml_path)
